In [41]:
import logging
import sys
import nest_asyncio

logging.basicConfig(stream=sys.stdout, level=logging.INFO)
logging.getLogger().addHandler(logging.StreamHandler(stream=sys.stdout))

nest_asyncio.apply()

In [42]:
from llama_index import (
    VectorStoreIndex,
    ServiceContext,
    Response,
    
)
import pinecone
import openai
import streamlit as st
from llama_index.vector_stores import PineconeVectorStore
from llama_index.llms import OpenAI
from llama_index.evaluation import RelevancyEvaluator
import pandas as pd

pd.set_option("display.max_colwidth", 0)

In [43]:
openai.api_key = st.secrets.openai_key
api_key = "2bcecfc8-35f9-44dc-9459-ab0cc2781b29"
pinecone.init(api_key=api_key, environment="gcp-starter")

In [44]:
pinecone_index = pinecone.Index("tailwind-hugging")
vector_store = PineconeVectorStore(pinecone_index=pinecone_index)
service_context = ServiceContext.from_defaults(llm=OpenAI(model="gpt-3.5-turbo", temperature=0.5, system_prompt="You are an expert on TailwindCSS and your job is to answer technical questions. Assume that all questions are related to TailwindCSS. Keep your answers technical and based on facts – do not hallucinate features."))
index = VectorStoreIndex.from_vector_store(vector_store=vector_store, service_context=service_context)

In [45]:
evaluator = RelevancyEvaluator(service_context=service_context)

In [46]:
def display_eval_df(query: str, response: Response, eval_result: str) -> None:
    eval_df = pd.DataFrame(
        {
            "Query": query,
            "Response": str(response),
            "Source": response.source_nodes[0].node.text[:1000] + "...",
            "Evaluation Result": "Pass" if eval_result.passing else "Fail",
        },
        index=[0],
    )
    eval_df = eval_df.style.set_properties(
        **{
            "inline-size": "600px",
            "overflow-wrap": "break-word",
        },
        subset=["Response", "Source"]
    )
    display(eval_df)

In [47]:
query_str = "How can I make an infinite scroll in TailwindCSS?"
query_engine = index.as_query_engine()
response_vector = query_engine.query(query_str)
eval_result_1 = evaluator.evaluate_response(
    query=query_str, response=response_vector
)


In [48]:
display_eval_df(query_str, response_vector, eval_result_1)

,Query,Response,Source,Evaluation Result
0,How can I make an infinite scroll in TailwindCSS?,"To create an infinite scroll in TailwindCSS, you can use the `overflow-y-auto` class on the container element. This will allow vertical scrolling if needed. Additionally, you can use the `h-72` class to set the height of the container element.","unsplash.com/photo-1605405748313-a416a1b84491?ixlib=rb-1.2.1&ixid=MnwxMjA3fDB8MHxwaG90by1wYWdlfHx8fGVufDB8fHx8&auto=format&fit=facearea&facepad=4&w=256&h=256&q=80"" /> Sarah ``` ```html ``` ### Scrolling vertically if needed Use `overflow-y-auto` to allow vertical scrolling if needed. ```html {{ example: true }} <stro...",Fail
